In [ ]:
import importlib, traceback, reluxury.pplx_client as pplx_client, inspect
print("module file:", pplx_client.__file__)
print("has PPLXClient?", "PPLXClient" in dir(pplx_client))
if "PPLXClient" not in dir(pplx_client):
    # 파일 내용을 실제로 확인 (앞 60줄)
    import pathlib
    src = pathlib.Path(pplx_client.__file__).read_text(encoding="utf-8")
    print("--- file head ---")
    print("\n".join(src.splitlines()[:60]))


In [ ]:
import pandas as pd

In [ ]:
# 데이터 불러오기

df_items = pd.read_csv('data/jewelry_items_word_with_category.csv')
df_sentiment = pd.read_csv('data/jewelry_sentiment_words.csv')
df_clean = pd.read_csv('data/cafe_posts_clean.csv')


In [ ]:
df_items.head()

In [ ]:
df_sentiment.head()

In [ ]:
df_clean.head()

In [ ]:
"""
pplx_client.py
Perplexity API 호출용 기본 클라이언트 (OpenAI SDK 호환)
"""

import os
import pandas as pd
from dotenv import load_dotenv
from openai import OpenAI


class PPLXClient:
    """Perplexity API 래퍼 클래스"""

    def __init__(self, model: str = "sonar"):
        load_dotenv()
        api_key = os.getenv("PPLX_API_KEY")
        if not api_key:
            raise ValueError("환경변수 PPLX_API_KEY를 설정해주세요.")

        self.client = OpenAI(api_key=api_key, base_url="https://api.perplexity.ai")
        self.model = model

    def ask(self, prompt: str, system_prompt: str = "Be precise and concise.") -> str:
        """단건 호출"""
        resp = self.client.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": prompt},
            ],
        )
        return resp.choices[0].message.content

    def ask_stream(self, prompt: str, system_prompt: str = "Be precise and concise."):
        """스트리밍 호출"""
        stream = self.client.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": prompt},
            ],
            stream=True,
        )
        for chunk in stream:
            delta = chunk.choices[0].delta
            if delta and delta.content:
                yield delta.content

    @staticmethod
    def preview_table(df: pd.DataFrame, n: int = 5) -> str:
        """데이터 미리보기 텍스트 생성"""
        return df.head(n).to_markdown(index=False)

    def ask_about_csv(self, df: pd.DataFrame, question: str) -> str:
        """CSV 미리보기 + 질문"""
        context = self.preview_table(df, n=5)
        prompt = (
            "당신은 초보 데이터 분석가입니다. 아래 표를 참고해 질문에 답하세요.\n\n"
            f"{context}\n\n질문: {question}"
        )
        return self.ask(prompt)


In [ ]:
%pip install -U openai python-dotenv pandas

In [ ]:
%pip install tabulate

In [ ]:
# 예시 코드 1
# # CSV 로드
df_items = pd.read_csv('data/jewelry_items_word_with_category.csv')

from reluxury.pplx_client import PPLXClient
client = PPLXClient(model="sonar")

print(client.ask("한 문장으로 자기소개 예시를 만들어줘.", temperature=0.2))
print(client.ask_about_df(df_items, "이 데이터의 주요 컬럼 3개만 뽑아 설명해줘."))
for tok in client.ask_stream("한국의 수도는?", temperature=0.0):
    print(tok, end="")


In [ ]:
# 예시 코드 2

import pandas as pd
from reluxury.pplx_client import PPLXClient

# 1) CSV 불러오기
df_items = pd.read_csv('data/jewelry_items_word_with_category.csv')
df_sentiment = pd.read_csv('data/jewelry_sentiment_words.csv')


# 2) 클라이언트 초기화
pplx = PPLXClient(model="sonar")

# 3) 일반 질문
print(pplx.ask("한 문장으로 자기소개 예시를 만들어줘."))

# 4) CSV 기반 질문
print(pplx.ask_about_df(df_items, "이 데이터의 주요 컬럼 3개만 뽑아 설명해줘."))

# 5) 스트리밍 출력
for token in pplx.ask_stream("스트리밍으로 한국의 수도를 알려줘."):
    print(token, end="", flush=True)


In [ ]:
# 예시 코드 3

import pandas as pd
from reluxury.pplx_client import PPLXClient

# CSV 로드
df_items = pd.read_csv('data/jewelry_items_word_with_category.csv')
df_sentiment = pd.read_csv('data/jewelry_sentiment_words.csv')

# 클라이언트
client = PPLXClient(model="sonar")  # 필요 시 "sonar-pro" 등

# 1) 단건 질문
print(client.ask("데이터 분석가의 역량 알려줘", temperature=0.7))

# 2) 메시지 배열
msgs = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Summarize the benefits of model quantization in 2 bullets."},
]
print(client.chat(msgs, temperature=0.3))

# 3) 스트리밍
for token in client.ask_stream("한국의 수도는?", temperature=0.0):
    print(token, end="", flush=True)

# 4) DataFrame 기반 Q&A
print(client.ask_about_df(df_items, "이 데이터의 주요 컬럼 3개만 뽑아 설명해줘."))


In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()  # .env 읽기
print("키 확인:", os.getenv("PPLX_API_KEY")[:8] if os.getenv("PPLX_API_KEY") else None)


In [ ]:
import os
print(os.getcwd())

In [ ]:
from dotenv import load_dotenv
load_dotenv(dotenv_path="/Users/t2024-m0246/Downloads/sparta-project/luxury/.env")


In [ ]:
import dotenv
print("dotenv 모듈 위치:", dotenv.__file__)
print("dotenv 모듈 버전:", dotenv.__version__)

In [ ]:
%pip show python-dotenv

In [ ]:
# 잘못된 dotenv 제거

%pip uninstall -y dotenv

In [ ]:
# 올바른 패키지 설치/업데이트

%pip install -U python-dotenv

In [ ]:
import importlib.metadata
print(importlib.metadata.version("python-dotenv"))

In [ ]:
pip install --upgrade pip

In [ ]:
%pip install -U python-dotenv

In [ ]:
import importlib.metadata
print(importlib.metadata.version("python-dotenv"))

세팅완료 !!!

In [ ]:
import pandas as pd
from reluxury.pplx_client import PPLXClient

# 1) CSV 불러오기
df_items = pd.read_csv('data/jewelry_items_word_with_category.csv')
df_sentiment = pd.read_csv('data/jewelry_sentiment_words.csv')
df_clean = pd.read_csv('data/cafe_posts_clean.csv')

# 2) 클라이언트 초기화
pplx = PPLXClient(model="sonar")

# 3) 일반 질문
# print(pplx.ask("한 문장으로 자기소개 예시를 만들어줘."))

# 4) CSV 기반 질문
# print(pplx.ask_about_df(df_items, df_sentiment, df_clean "")) # 이러면 에러남.
# df 파일 하나씩 넣거나 통합된 DataFrame으로 넣어줘야 함. 

# # 질문예시 1. df_items (브랜드/라인 정보)
# print(pplx.ask_about_df(df_items, "브랜드명이 IWC인 제품 라인 이름들을 알려줘."))
# print(pplx.ask_about_df(df_items, "relative_word 컬럼에 등록된 별칭이 가장 많은 브랜드는 어디야?"))

# 질문예시 2. df_sentiment (감성 단어 사전)
# print(pplx.ask_about_df(df_sentiment, "긍정 카테고리에 해당하는 단어들은 몇 개인지 알려줘."))
# print(pplx.ask_about_df(df_sentiment, "부정 카테고리에 속한 단어 예시 5개만 보여줘."))

# 질문예시 3. df_clean (카페 게시글)
# print(pplx.ask_about_df(df_clean, "조회수가 가장 높은 게시글의 제목을 알려줘."))
print(pplx.ask_about_df(df_clean, "댓글 수가 많은 게시글의 제목 3개만 뽑아줘."))

# 5) 스트리밍 출력
# for token in pplx.ask_stream("스트리밍으로 한국의 수도를 알려줘."):
#     print(token, end="", flush=True)

In [ ]:
df_items, df_sentiment, df_clean 를 함께 활용해서 → 긍정 단어가 가장 많이 나온 제품명을 찾기

In [ ]:
df_items.head()

In [ ]:
df_sentiment.head()

In [ ]:
df_clean.head()

In [ ]:
# 시간 오래 걸림 주의 (필자는 1m57.5s 소요됨)

import pandas as pd
import numpy as np
import re

# ---------- 0) 유틸 ----------
def split_csv_list(s: str) -> list[str]:
    """쉼표로 구분된 문자열을 리스트로. 공백/빈값/NaN 안전 처리."""
    if pd.isna(s):
        return []
    return [x.strip() for x in str(s).split(",") if x.strip()]

def safe_join_name(brand, line):
    """브랜드/라인 조합으로 제품명 라벨 생성"""
    b = str(brand).strip() if pd.notna(brand) else ""
    l = str(line).strip() if pd.notna(line) else ""
    return f"{b} {l}".strip() if l else b

# ---------- 1) 긍정 단어 사전 구축 ----------
# df_sentiment: [category, words]
mask_pos = df_sentiment["category"].astype(str).str.endswith("긍정")
pos_words = []
for words in df_sentiment.loc[mask_pos, "words"].dropna():
    pos_words.extend(split_csv_list(words))

# 중복 제거 & 길이 내림차순(긴 단어를 먼저 매칭해 부분중복 영향↓)
pos_words = sorted(set(pos_words), key=len, reverse=True)
# 긍정 단어 정규식 (특수문자 이스케이프)
pos_re = re.compile("|".join(map(re.escape, pos_words))) if pos_words else None

# ---------- 2) 제품(별칭) 테이블 준비 ----------
# df_items: [brand_id, brand_name, line_id, line_name, category_id, category_name, class_type, relative_word]
items = df_items.copy()

# 제품 라벨(출력용)
items["product_label"] = items.apply(lambda r: safe_join_name(r.get("brand_name"), r.get("line_name")), axis=1)

# 별칭 리스트: relative_word + brand_name + line_name
def build_aliases(row) -> list[str]:
    aliases = set(split_csv_list(row.get("relative_word")))
    if pd.notna(row.get("brand_name")) and str(row.get("brand_name")).strip():
        aliases.add(str(row.get("brand_name")).strip())
    if pd.notna(row.get("line_name")) and str(row.get("line_name")).strip():
        aliases.add(str(row.get("line_name")).strip())
    # 길이 내림차순 (긴 별칭 우선 매칭)
    return sorted({a for a in aliases if a}, key=len, reverse=True)

items["aliases"] = items.apply(build_aliases, axis=1)

# 별칭 정규식(제품별)
def compile_alias_re(aliases):
    if not aliases:
        return None
    return re.compile("|".join(map(re.escape, aliases)))
items["alias_re"] = items["aliases"].apply(compile_alias_re)

# ---------- 3) 콘텐츠 합치기 ----------
# df_clean: [title, text, comments, ...]
clean = df_clean.copy()
for c in ["title", "text", "comments"]:
    if c not in clean.columns:
        clean[c] = ""
clean["content"] = (
    clean["title"].fillna("").astype(str) + "\n" +
    clean["text"].fillna("").astype(str) + "\n" +
    clean["comments"].fillna("").astype(str)
)

contents = clean["content"].tolist()  # 속도 때문에 리스트로 빼두기

# ---------- 4) 제품별 긍정 단어 카운트 ----------
results = []
for idx, row in items.iterrows():
    alias_re = row["alias_re"]
    if alias_re is None or pos_re is None:
        pos_count = 0
        doc_hits = 0
    else:
        pos_count = 0
        doc_hits = 0
        for doc in contents:
            if alias_re.search(doc):              # 제품 언급된 글만 집계
                doc_hits += 1
                pos_count += len(pos_re.findall(doc))
    results.append({
        "brand_id": row.get("brand_id"),
        "product_label": row["product_label"],
        "positive_word_count": int(pos_count),
        "mentioned_docs": int(doc_hits),
        "aliases": row["aliases"],
        "category_name": row.get("category_name"),
        "line_name": row.get("line_name"),
    })

result_df = pd.DataFrame(results)

# ---------- 5) 상위 결과 확인 ----------
topN = (
    result_df.sort_values(["positive_word_count", "mentioned_docs"], ascending=[False, False])
    .head(5)
    .reset_index(drop=True)
)

print("✅ 긍정 단어가 많이 언급된 상위 TOP5 제품")
print(topN[["product_label", "positive_word_count", "mentioned_docs"]].to_string(index=False))


In [ ]:
df_items = pd.read_csv("data/1.jewelry_items_word_with_category.csv")

In [ ]:
df_items.head()

In [ ]:
import pandas as pd
import re

# 1) 정규화 사전 불러오기
df_items = pd.read_csv("data/1.jewelry_items_word_with_category.csv")  # variant, canonical
replace_map = dict(zip(df_items["variant"], df_items["canonical"]))

# 2) 치환 함수
def normalize_text(text, mapping):
    # 긴 단어부터 치환 (부분 매칭 방지)
    pairs = sorted(mapping.items(), key=lambda kv: len(kv[0]), reverse=True)
    for variant, canon in pairs:
        text = re.sub(re.escape(variant), canon, text)
    return text

# 예시 텍스트
raw = "이 반지 진짜 예쁘네, 이쁘고 착용감도 좋아요!"
norm = normalize_text(raw, replace_map)
print("정규화 전:", raw)
print("정규화 후:", norm)

# 3) LLM 호출
from reluxury.pplx_client import PPLXClient
client = PPLXClient(model="sonar")

print(client.ask(f"아래 텍스트의 감성을 요약해줘:\n{norm}"))


정규화된 별도 파일이 필요함.
왜냐하면 "variant" 해당하는 컬럼과 "canonical" 에 해당하는 컬럼으로 구분해서 LLM에 전달해야하는데 

예를 들어 df_items 에서 "variant"는 "relative_word"가 되고
"canonical" 이라는 대표성을 띄는 컬럼이 필요해. 

그래서 "brand_id" + "line_id" + "cattegory_id" 를 합친 컬럼을 하나 생성할거야. 

그런데 해당 파일에 공란들이 있어. 
이 공란을 메꾸는 작업이 1 첫번째 수행작업임. 

=> 우선은 공란 메꾸기 보다 
불가리 제품의 인기템 3가지 디바스드림, 비제로원, 세르펜디 3가지만 가지고 해보자. 

In [ ]:
df_items.isnull().sum()

In [ ]:
df_sentiment.isnull().sum()

In [ ]:
# CSV 불러오기
import pandas as pd 
df_items = pd.read_csv("data/1.jewelry_items_word_with_category.csv")
df_sentiment = pd.read_csv("data/2.jewelry_sentiment_words.csv")
df_place = pd.read_csv('data/3.jewelry_place_word_with_category.csv')
df_place_sentiment = pd.read_csv('data/4.place_sentiment_keywords.csv')


In [ ]:
df_place.isnull().sum()

In [ ]:
df_place_sentiment.isnull().sum()

In [ ]:
# df_items 파일에서 불가리 인기템 3가지 라인 추출하자.

import pandas as pd

# 파일 불러오기
df_items = pd.read_csv("data/1.jewelry_items_word_with_category.csv")

# 조건에 맞는 line_id만 필터링
target_ids = ["l020", "l055", "l064"]
filtered = df_items[df_items["line_id"].isin(target_ids)]

print(filtered.head())
print("총 행 개수:", len(filtered))

# 필요하다면 새로운 CSV로 저장
filtered.to_csv("data/Bulgary_top3_items_filtered.csv", index=False)